In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [12]:
def T(a_arr, alpha_arr, d_arr, theta_arr):
    pose = np.identity(4)
    for i in range(len(theta_arr)):
        a, alpha, d, theta = a_arr[i], alpha_arr[i], d_arr[i], theta_arr[i]
        rot = np.matrix([
            [np.cos(theta), -np.sin(theta), 0, 0],
            [np.sin(theta),  np.cos(theta), 0, 0],
            [0            ,              0, 1, d],
            [0            ,              0, 0, 1],
        ])
        trans = np.matrix([
            [1,             0,              0, a],
            [0, np.cos(alpha), -np.sin(alpha), 0],
            [0, np.sin(alpha),  np.cos(alpha), 0],
            [0,             0,              0, 1],
        ])
        pose = np.matmul(pose, np.matmul(rot, trans))
    return pose

In [167]:
def z(i_minus_1, theta_arr, alpha_arr): # returns z_{i-1}.
    #z_0 = np.array([[0,0,1]]).T
    if i_minus_1==0:
        return np.array([0,0,1]).T
    R = np.identity(3)
    for x in range(1, i_minus_1+1): # x in theta_arr:
        theta, alpha = theta_arr[x-1], alpha_arr[x-1]
        R_z = np.matrix([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [            0,              0, 1]
        ])
        R_x = np.matrix([
            [1,             0,              0],
            [0, np.cos(alpha), -np.sin(alpha)],
            [0, np.sin(alpha),  np.cos(alpha)],
        ])
        R = np.matmul(R, np.matmul(R_z, R_x))
    return np.asarray(R[:,2].T)[0].T # third column

In [168]:
z(2, [np.pi/2, -np.pi/2], [np.pi/4, np.pi/2])

array([ 2.53632657e-17, -1.00000000e+00,  9.81389106e-33])

In [169]:
def p(i_minus_1, a_arr, alpha_arr, d_arr, theta_arr):
    a, alpha, d, theta = a_arr[:i_minus_1+1], alpha_arr[:i_minus_1+1], d_arr[:i_minus_1+1], theta_arr[:i_minus_1+1]
    #a, alpha, d, theta = a_arr[i_minus_1], alpha_arr[i_minus_1], d_arr[i_minus_1], theta_arr[i_minus_1]
    pose = T(a, alpha, d, theta)
    return np.asarray(pose[:3,-1].T)[0].T

In [174]:
def jacobian(a_arr, alpha_arr, d_arr, theta_arr):
    I = len(theta_arr)
    p_e = p(I-1, a_arr, alpha_arr, d_arr, theta_arr)
    J = np.zeros(shape=(6,I))
    for i in range(I):
        z_i = z(i, theta_arr, alpha_arr)
        diff = p_e - p(i, a_arr, alpha_arr, d_arr, theta_arr)
        cp = np.cross(z_i.T, diff.T)
        col = np.concatenate((cp, z_i))
        print col
        J[:,i] = col
    return J

In [175]:
a_arr = [0, 0, 0, 0.0825, -0.0825, 0, 0.088, 0]
alpha_arr = [0, -np.pi/2, np.pi/2, np.pi/2, -np.pi/2, np.pi/2, np.pi/2, 0]
d_arr = [0.333, 0, 0.316, 0, 0.384, 0, 0, 0.107]
theta_arr = [0.1, 0.2, 0.3, -0.4, 0.5, 0.6, 0.7]
jacobian(a_arr, alpha_arr, d_arr, theta_arr)

[ 0.04093309 -0.081512    0.          0.          0.          1.        ]
[ 0.04093309 -0.081512    0.          0.          0.          1.        ]
[ 6.34180589e-02  1.96175045e-02  8.99679382e-02 -2.95520207e-01
  9.55336489e-01  6.12323400e-17]
[ 0.32643983 -0.08906299 -0.08832786  0.28232124  0.08733219  0.95533649]
[-0.06690997  0.01043802  0.04139816 -0.08321745 -0.98986424  0.11508099]
[-0.03239536  0.05713095 -0.014721   -0.21042857  0.1303323   0.9688825 ]
[-0.          0.          0.          0.48131384 -0.8488226   0.21871712]


array([[ 4.09330874e-02,  4.09330874e-02,  6.34180589e-02,
         3.26439829e-01, -6.69099747e-02, -3.23953636e-02,
        -0.00000000e+00],
       [-8.15119957e-02, -8.15119957e-02,  1.96175045e-02,
        -8.90629910e-02,  1.04380156e-02,  5.71309495e-02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  8.99679382e-02,
        -8.83278623e-02,  4.13981578e-02, -1.47209988e-02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.95520207e-01,
         2.82321237e-01, -8.32174494e-02, -2.10428570e-01,
         4.81313842e-01],
       [ 0.00000000e+00,  0.00000000e+00,  9.55336489e-01,
         8.73321925e-02, -9.89864244e-01,  1.30332302e-01,
        -8.48822602e-01],
       [ 1.00000000e+00,  1.00000000e+00,  6.12323400e-17,
         9.55336489e-01,  1.15080989e-01,  9.68882505e-01,
         2.18717116e-01]])